# Real World Example!

Wenn man nur die Grundkonzpete gesehen hat, dann wird es auf Anhieb nicht klar, was man mit diesem Framework anfangen kann. Deswegen wollen wir uns ein einfaches Beispiel ansehen, das einen echten Usecase darstellen kann.
Und wie wir gesehen haben, handelt es sich um ein Workflow-System. Deswegen wird das Beispiel natürlich einen Workflow nutzen...

Das Szenario: Wir machen einen KI-Chatbot für ein Restaurant. Dieser Chatbot soll Anfragen in natürlicher Sprache entgegen nehmen. Über den Chatbot kann der Benutzer Bestellungen abgeben oder Informationen wie z.B. Öffnungszeiten abrufen.

Lösung: Wir nutzen ein Agenten-System. Erstmal brauchen wir einen Agenten, der die Aufgabe hat die Absicht des Benutzers erkennt. Wir machen uns das Leben hier einfach, d.h. erlaubt ist entweder bestellen oder Infos abrufen.
Wenn der User Infos haben möchte, dann routen wir zu dem Info-Assistenten, der die Anfrage beantwortet.

Im Falle einer Bestellung müssen wir prüfen, ob wir alle Angaben haben. Wir müssen zum einen wissen, was der Benutzer bestellen möchte und außerdem wohin das Essen geliefert werden soll. Hierfür nutzen wir auch einen Agenten zur Prüfung. Falls Angaben fehlen, wird der Benutzer danach gefragt.

Wenn alles OK ist, dann wird der Bestell-Agent beauftrag die Bestellung für die Küche aufzubereiten. Dazu wird er ein Tool nutzen. In einer echten Anwendung wäre das z.B. eine Datenbank. Wir machen es hier aber recht einfach, in dem das Tool eine formatierte Ausgabe macht, in der alle wichtigen Angaben für die Küche enhalten sind.

Zu guter Letzt wird der Auftrag bestätigt und der Prozess ist beendet.

Schauen wir uns das als Diagramm mal an...

![Diagramm](./bilder/agent-flow.png)

Lasst uns die Anwendung sukzessive aufbauen. Wir fangen erstmal mit der üblichen Initialisierung und Installation an. Wir legen dann ein Datenmodell (_State_) an.
Dann legen wir jeden Agenten als Funktion (_Node_) an. Wir legen ausserdem das Tool an.

Zuletzt erstellen wir den Graph und verknüpfen alles zu einem Gesamtsystem.

In [ ]:
!pip install langgraph langchain_openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")

# model = ChatOpenAI(model="gpt-4o")
model = AzureChatOpenAI(openai_api_version="2024-05-01-preview", azure_deployment="gpt-4o", temperature=0.5)

#model = ChatOpenAI(model="gpt-4o")

In [ ]:
# State
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    missing_info: str

In [ ]:
def ask_user(state: State):
    # Hier fragen wir den Nutzer, was er tun möchte
    user_input = input("Wilkommen! Bitte geben Sie hier Ihr Anliegen ein: ")
    state["messages"].append(HumanMessage(user_input))
    return state

In [ ]:
# Intent-Agent
intent_agent_system_template = """Du bist ein Agent eines Restaurant-Bestell-Systems. Deine Aufgabe ist es die Absicht des Nutzers zu erkennen. 
Der Nutzer kann zwei Absichten haben: eine Bestellung aufgeben oder Informationen zu dem Restaurant erhalten.
Bitte erkenne die Absicht des Nutzers und gib sie an. Die möglichen Absichten sind:
'info' oder 'order'. Antworte nur mit 'info' oder 'order'.
"""

intent_agent_prompt_template = ChatPromptTemplate.from_messages(
    [("system", intent_agent_system_template), ("user", "Hier ist die Anfrage des Nutzers: {text}")]
)

intent_agent_chain = intent_agent_prompt_template | model | StrOutputParser()

def intent_agent(state: State):
    # Wir beauftragen die KI die Absicht des Nutzers zu erkennen: "info" oder "order"
    result = intent_agent_chain.invoke(state["messages"])
    print(f"Intent: {result}")
    return result

In [ ]:
info_agent_system_template = """Du bist ein Agent eines Restaurant-Bestell-Systems. Deine Aufgabe ist es Informationen über das Restaurant zu geben. 
Bitte gib die Informationen an, die der Nutzer benötigt.

Restaurant-Informationen:
- Name: Restaurant "Zur wildgewordenen KI"
- Adresse: KI-Straße 42, 12345 KI-Stadt
- Telefonnummer: 01234 567890
- Öffnungszeiten: Montag bis Freitag von 10 bis 22 Uhr
- Speisekarte: Pizza, Pasta, Salate, Desserts
- Lieferung: Ja, Lieferung möglich
- Reservierung: Ja, Reservierung möglich
- Sonstiges: WLAN, Barrierefreiheit
"""

info_agent_prompt_template = ChatPromptTemplate.from_messages(
    [("system", info_agent_system_template), ("user", "Hier ist die Anfrage des Nutzers: {text}")]
)

info_agent_chain = info_agent_prompt_template | model | StrOutputParser()

def info_agent(state: State):
    # Dieser Agent beantwortet die Frage. Der Agent hat die notwendigen Informationen in seinem System-Prompt, um die Frage zu beantworten.
    result = info_agent_chain.invoke(state["messages"])
    print(f"Info: {result}")

    state["messages"].append(AIMessage(result))
    return state

In [ ]:

def accept_order_agent(state: State):
    print("Bestellung annehmen")
    # Dieser Agent nimmt die Bestellung entgegen. Hier passiert erstmal nichts. In einer echten Anwendung würde hier die Bestellung gespeichert.
    return state

In [ ]:
order_check_agent_system_template = """Du bist ein Agent eines Restaurant-Bestell-Systems. Deine Aufgabe ist es die Bestellung des Kunden auf Vollständigkeit zu prüfen.
Zu einer Bestellung gehören folgende Informationen:
- Was soll bestellt werden?
- Wohin soll die Bestellung geliefert werden?
- Wann soll die Bestellung geliefert werden?

Wenn etwas fehlt, dann formuliere eine Frage, die den Nutzer dazu auffordert, die fehlenden Informationen anzugeben.
Wenn nichts fehlt, antworte mit 'Alles vollständig'.
"""

order_check_prompt_template = ChatPromptTemplate.from_messages(
    [("system", order_check_agent_system_template), ("user", "Hier ist die Bestellung des Nutzers: {text}")]
)

order_check_chain = order_check_prompt_template | model | StrOutputParser()

def order_check_agent(state: State):
    print("Bestellung überprüfen")
    # Dieser Agent überprüft, ob alle Angaben für die Bestellung vorhanden sind. Wenn nicht, dann antwortet er mit "incomplete". Wenn ja, dann antwortet er mit "complete".
    # Die Info, die noch fehlt, wird in der state gespeichert.

    result = order_check_chain.invoke(state["messages"])
    print(f"Order check: {result}")

    if result.startswith("Alles vollständig"):
        return "complete"

    state["missing_info"] = result

    return "incomplete"

In [ ]:
def missing_info_agent(state: State):
    print("Fehlende Information anfordern")
    # Dieser Agent fragt nach der fehlenden Information.
    # Hier fragen wir den Nutzer, was er tun möchte
    user_input = input(f"Es tut mir leid aber folgende Information fehlt noch: {state['missing_info']}: ")
    state["messages"].append(HumanMessage(user_input))

    return state

In [ ]:
from langchain_core.tools import tool

order_create_system_template = """Du bist ein Agent eines Restaurant-Bestell-Systems. Deine Aufgabe ist es die Bestellung im Restaurant-Bestell-System zu erstellen.
"""

order_create_prompt_template = ChatPromptTemplate.from_messages(
    [("system", order_create_system_template), ("user", "Erstelle die Bestellung mit den folgenden Informationen: {text}")]
)

order_create_chain = order_create_prompt_template | model | StrOutputParser()

@tool
def create_order(meal_orders: list[str], delivery_address: str, delivery_time: str):
    """Erstellt eine Bestellung im Restaurant-Bestell-System."""

    print("Bestellung:")
    print(f"Gerichte: {meal_orders}")
    print(f"Lieferadresse: {delivery_address}")
    print(f"Lieferzeit: {delivery_time}")

    return "Bestellnummer: 47110815"


tools = [create_order]

model.bind_tools(tools)

def create_order_agent(state: State):
    print("Bestellung erstellen")
    # Dieser Agent erstellt die Bestellung mit dem Tool.

    result = order_create_chain.invoke(state["messages"])
    print(f"Order create: {result}")

    return state

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)

graph_builder.add_node("ask", ask_user)
graph_builder.add_node("info", info_agent)
graph_builder.add_node("accept_order", accept_order_agent)
graph_builder.add_node("missing_info_agent", missing_info_agent)
graph_builder.add_node("create_order", create_order_agent)

graph_builder.add_edge(START, "ask")
graph_builder.add_conditional_edges("ask", intent_agent, { "info": "info", "order": "accept_order" })
graph_builder.add_conditional_edges("accept_order", order_check_agent, { "complete": "create_order", "incomplete": "missing_info_agent" })
graph_builder.add_conditional_edges("missing_info_agent", order_check_agent, { "complete": "create_order", "incomplete": "missing_info_agent" })
graph_builder.add_edge("create_order", END)
graph_builder.add_edge("info", END)

graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"messages": []})